In [1]:
import matplotlib.pyplot as plt
from imblearn.datasets import make_imbalance
from sklearn.metrics import classification_report
import numpy as np
from scipy.io import arff

from qore_sdk.utils import sliding_window
from qore_sdk.featurizer import Featurizer
from qore_sdk.client import WebQoreClient

from dataloader import load_dataset

# 取得したアカウント情報の読み込み
from credentials import username, password, endpoint

%matplotlib inline

In [31]:
# ref. http://timeseriesclassification.com/description.php?Dataset=SmallKitchenAppliances
dataset_train, meta_train = arff.loadarff("dataset/SmallKitchenAppliances_TRAIN.arff")
dataset_test, meta_test = arff.loadarff("dataset/SmallKitchenAppliances_TEST.arff")

In [153]:
# Kettle, Microwave and Toaster
ds_train = np.asarray(dataset_train.tolist(), dtype=np.float32)
ds_test = np.asarray(dataset_test.tolist(), dtype=np.float32)
X_train = ds_train[:, 0:-1]
X_test = ds_test[:, 0:-1]
y_train = ds_train[:, -1]
y_test = ds_test[:, -1]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
# print(ds[:, -1])
# i = 1
# plt.ylim(ymin=-0.07715, ymax=-0.078) 
# print(ds[i][0:-1])
# plt.plot(ds[i][0:-1])
# print(ds[i][-1])

(375, 720)
(375, 720)
(375,)
(375,)


In [154]:
# 前処理
# 正規化とか標準化してみるか
# ref. https://qiita.com/ttskng/items/2a33c1ca925e4501e609
# plt.yscale('log')
# plt.plot((X_train[i] + 1) / X_train[i])
X_train = X_train + 1
X_test = X_test + 1

In [155]:
width = 144
stepsize = 2
n_filters = 10

# ここで(サンプル数, 時系列長, 小時系列長)の配列を作成
X_train2 = sliding_window(X_train, width, stepsize)
X_test2 = sliding_window(X_test, width, stepsize)
print('X_train2.shape:', X_train2.shape)
print('X_test2.shape:', X_test2.shape)

# (サンプル数, 時系列長, 変量数)の配列に変換(Qoreへの入力となる)
featurizer = Featurizer(n_filters)
X_train2 = featurizer.featurize(X_train2, axis=2)
X_test2 = featurizer.featurize(X_test2, axis=2)
print(X_train2.shape, X_test2.shape)

X_train2.shape: (375, 289, 144)
X_test2.shape: (375, 289, 144)
(375, 289, 10) (375, 289, 10)


In [156]:
# 学習
client = WebQoreClient(username, password, endpoint=endpoint)
time_ = client.classifier_train(X_train2, y_train)
print('Time:.', time_, 'min.')

Time:. {'res': 'ok', 'train_time': 7.7187535762786865} min.


In [157]:
# 予測
res = client.classifier_predict(X_test2)

In [158]:
# 評価
report = classification_report(y_test, res['Y'])
print(report)

              precision    recall  f1-score   support

         1.0       0.82      0.82      0.82       125
         2.0       0.78      0.72      0.75       125
         3.0       0.71      0.77      0.74       125

    accuracy                           0.77       375
   macro avg       0.77      0.77      0.77       375
weighted avg       0.77      0.77      0.77       375

